## 🚀 Lead Enrichment AI Workflow

In this workflow, you'll provide a company name and website. Our AI will:

1. Explore the company's website and search the web for the latest news and activities.
2. Score the lead using your Ideal Customer Profile (ICP) and the gathered insights.
3. Decide whether a proposal should be sent.
4. If qualified, automatically generate and send a tailored proposal via email.

# <img src="./data/lead_enrichment_graph.png" alt="Agentic RAG" width="600">

In [ ]:
# Add necessary imports here

from dotenv import load_dotenv
from langgraph.graph import StateGraph, START, END
from langchain.agents import Tool
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
import nest_asyncio
from pydantic import BaseModel, Field
from typing import Annotated, List, TypedDict
from enum import Enum
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langchain_openai import ChatOpenAI
from datetime import datetime
import resend
import os

load_dotenv(override=True)
resend.api_key = os.getenv("RESEND_API_KEY")
nest_asyncio.apply()

In [ ]:
# Add state class and pydantic models for state and structured output

class LeadCategory(str, Enum):
    disqualified = "DISQUALIFIED"
    cold = "COLD"
    warm = "WARM"
    hot = "HOT"

class SuggestedService(BaseModel):
    name: str = Field(description="The name of the service")
    rationale: str = Field(description="The rationale for the suggested service")

class LeadScoringOutput(BaseModel):
    lead_score: int = Field(description="The score of the lead between 0 and 5")
    likelihood_score: int = Field(description="The likelihood score of the lead between 0 and 5")
    category: LeadCategory = Field(description="The category of the lead")
    suggested_services: List[SuggestedService] = Field(description="The top 2-3 services that the company is most likely to need")
    lead_score_reasoning: str = Field(description="The reasoning behind the lead score and category")

class State(TypedDict):
    messages: Annotated[list, add_messages]
    website_research_messages: Annotated[list, add_messages]
    news_research_messages: Annotated[list, add_messages]
    website: str
    website_research: str
    company: str
    news_research: str
    evaluated_lead_score: int
    evaluated_lead_likelihood_score: int
    evaluated_lead_category: LeadCategory
    evaluated_lead_services: List[SuggestedService]
    evaluated_lead_score_reasoning: str

In [ ]:
# Email tool setup

@tool
def email_tool(subject: str, html_body: str):
    """
    Sends an email using the Resend API.

    Args:
        subject (str): The subject of the email.
        html_body (str): The HTML content of the email body.

    Returns:
        str: Confirmation message indicating successful sending of the email.
    """

    params: resend.Emails.SendParams = {
        "from": "onboarding@resend.dev",
        "to": "murtaza.khan@bytemage.io",
        "subject": subject,
        "html": html_body,
    }
    resend.Emails.send(params)
    return "EMAIL SENT SUCCESSFULLY"

In [ ]:
# Search Tool Setup
search = GoogleSerperAPIWrapper()
search_tool = Tool(name="tool_search", func=search.run, description="useful when you need to search the internet for information")

In [ ]:
# PlayWright Tools Setup
async_browser = create_async_playwright_browser(headless=False)
browser_toolkit = PlayWrightBrowserToolkit(async_browser=async_browser)
browser_tools = browser_toolkit.get_tools()

In [ ]:
# LLM Definitions binded with tools

website_research_tools = browser_tools
website_research_llm = ChatOpenAI(model="gpt-4o-mini").bind_tools(website_research_tools)
website_research_tool_node = ToolNode(tools=website_research_tools, messages_key="website_research_messages")

news_research_tools = [search_tool] + browser_tools
news_research_llm = ChatOpenAI(model="gpt-4o-mini").bind_tools(news_research_tools)
news_research_tool_node = ToolNode(tools=news_research_tools, messages_key="news_research_messages")

scoring_llm = ChatOpenAI(model="gpt-4o-mini").with_structured_output(LeadScoringOutput)

proposal_llm = ChatOpenAI(model="gpt-4o-mini").bind_tools([email_tool])
proposal_tool_node = ToolNode(tools=[email_tool])

In [ ]:
# Add website research node
# This node will be responsible for going through the website and extracting relevant information

def website_research_node(state: State) -> State:
    """
    This node is responsible for researching the website and returning the results.
    """
    website = state["website"]

    instructions = f"""
    You are an experienced sales development representative working for a software development services company and prospecting for new clients.
    You are given a website link and you need to research the website and see if the company is a good fit for software development services.
    You look at the website and see what the company does, what they are known for, what they are looking for, and what they might need help with.
    Navigate to homepage, about page, jobs page, news page and team page if available to get a holistic view of the company.
    You then provide a summary of your findings to the user as markdown.

    Website: {website}
    """

    result = website_research_llm.invoke([{"role": "user", "content": instructions}] + state["website_research_messages"])

    new_state = {"website_research_messages": [result]}
    
    return new_state

In [ ]:
# Add news research node
# This node will be responsible for searching latest news about the provided company on the internet

def news_research_node(state: State) -> State:
    """
    This node is responsible for researching about latest news and events of the company using web search and browser tools by visiting different news websites.
    """
    company = state["company"]
    website = state["website"]

    instructions = f"""
    You are an experienced sales development representative working for a software development services company and prospecting for new clients.
    You are given a company name and websiteand you need to research about latest news and events of the company using web search and browser tools by visiting different news websites.
    You search for news and events using web search tool and then visit different news websites to get a holistic view of the company.
    You then provide a summary of your findings to the user as markdown.

    Company Name: {company}
    Website: {website}
    Current Date: {datetime.now().strftime("%Y-%m-%d")}
    """

    result = news_research_llm.invoke([{"role": "user", "content": instructions}] + state["news_research_messages"])

    new_state = {"news_research_messages": [result]}
    
    return new_state


In [ ]:
# Define prompt for providing LLM with ICP and instructions for scoring leads

def scoring_instructions(website_research: str, news_research: str) -> str:
    return f"""
You are a lead research and scoring assistant for ByteMage, a software development company. 
Your task is to analyze research data about a company and score it according to ByteMage’s services and Ideal Customer Profile (ICP).

## About ByteMage
- Core services: End-to-end custom software development (web apps, mobile apps, enterprise systems).
- AI focus: Building AI-powered applications and AI agents; enabling businesses with AI transformation.
- Engagement model: Flexible (end-to-end delivery or tailored to client needs).
- Strengths: Blend of passionate technologists and business expertise.

## Companies ByteMage can serve
- Startups scaling up, and mid-market enterprises undergoing digital or AI transformation.
- Not a fit: very large corporations (billion-dollar scale), defense/military companies, or organizations with no real software/technical needs.
- Geographic focus: US and Europe.

## Ideal Customer Profile (ICP)
- **Business stage:** Startups in scaling phase, enterprises needing transformation (especially AI enablement).
- **Buyer roles:** Founders, CEOs, CTOs, Directors of IT, Heads of Digital Transformation.
- **Pain points:** 
    - Struggle to find right expertise/talent.
    - Need to scale existing systems.
    - Desire to build new (greenfield) projects.
    - Need to implement AI features/agents into existing systems.
- **Buying signals:** 
    - Recent funding.
    - Hiring for technical roles.
    - Public AI adoption plans.
    - Digital transformation initiatives.

## Scoring Instructions
1. **Lead Score (0–5):** 
    - How desirable this company is for ByteMage based on ICP alignment, size, stage, geography, and needs.
    - High score = strong ICP alignment + clear software/AI needs.

2. **Likelihood Score (0–5):**
    - How likely ByteMage is to realistically win this client given its size, budget, competition, and ByteMage’s current maturity.
    - High score = within ByteMage’s reach (not too big, not too small, and realistic to win).

3. Provide a **Reasoning Summary**:
    - 3–5 concise bullet points explaining why you gave these scores.
    - Highlight key ICP matches, pain points, or buying signals.

4. Suggest **Relevant Services**:
    - Top 2–3 ByteMage services the company is most likely to need (e.g., web app dev, mobile dev, AI agent implementation).
    - Include a short rationale for each.

## Input
You will receive:
- Company website text (scraped)
- Internet search results (news, events, funding, hiring)

Analyze this input against ByteMage’s profile and output the JSON result.

Current Date: {datetime.now().strftime("%Y-%m-%d")}

Website Research:
{website_research}

News Research:
{news_research}
    """

In [ ]:
# Add lead scoring node
# This node will be responsible for digesting the news and website research and the provided ICP and score the lead

def lead_scoring_node(state: State) -> State:
    """
    This node is responsible for scoring the lead and returning the results.
    """
    website_research = state["website_research_messages"][-1].content
    news_research = state["news_research_messages"][-1].content

    result = scoring_llm.invoke([{"role": "system", "content": scoring_instructions(website_research, news_research)}])
    return {
        "evaluated_lead_score": result.lead_score,
        "evaluated_lead_likelihood_score": result.likelihood_score,
        "evaluated_lead_category": result.category,
        "evaluated_lead_services": result.suggested_services,
        "evaluated_lead_reasoning": result.lead_score_reasoning
    }

In [ ]:
# Add send proposal node
# This node will be responsible for crafting a business proposal and emailing it

def send_proposal_node(state: State):
    instructions = f"""
    You are an experienced proposal writer and sales development representative working for a software development services company and prospecting for new clients.
    You are provided with the company name, company news, research about company's website and potential services that the company might need.
    You need to write a proposal for the company and email it to the company.

    Company Name: {state["company"]}
    
    Company News:
    {state["news_research_messages"][-1].content}
    
    Company Website Research:
    {state["website_research_messages"][-1].content}
    
    Potential Services:
    {"\n".join([f"- {service.name}: {service.rationale}" for service in state["evaluated_lead_services"]])}
    """
    result = proposal_llm.invoke([{"role": "system", "content": instructions}] + state["messages"])

    return {"messages": [result]}

In [ ]:
# Add router for deciding to send proposal or not

def decision_router(state: State):
    if state["evaluated_lead_score"] >= 3:
        return "send_proposal"
    return "dont_send_proposal"

# Add custom tools condition function since we need to provide the tools condition with custom messages key

def website_research_tools_condition(state):
    return tools_condition(state, "website_research_messages")

def news_research_tools_condition(state):
    return tools_condition(state, "news_research_messages")

In [ ]:
# Add nodes and edges to the graph as defined in the illustration

graph_builder = StateGraph(State)

graph_builder.add_node("website_research_node", website_research_node)
graph_builder.add_node("news_research_node", news_research_node)
graph_builder.add_node("lead_scoring_node", lead_scoring_node, defer=True)
graph_builder.add_node("website_research_tool_node", website_research_tool_node)
graph_builder.add_node("news_research_tool_node", news_research_tool_node)
graph_builder.add_node("send_proposal_node", send_proposal_node)
graph_builder.add_node("proposal_tool_node", proposal_tool_node)

graph_builder.add_edge(START, "website_research_node")
graph_builder.add_edge(START, "news_research_node")

graph_builder.add_conditional_edges("website_research_node", website_research_tools_condition, {"tools": "website_research_tool_node", END: "lead_scoring_node"})
graph_builder.add_edge("website_research_tool_node", "website_research_node")

graph_builder.add_conditional_edges("news_research_node", news_research_tools_condition, {"tools": "news_research_tool_node", END: "lead_scoring_node"})
graph_builder.add_edge("news_research_tool_node", "news_research_node")

graph_builder.add_conditional_edges("lead_scoring_node", decision_router, {"send_proposal": "send_proposal_node", "dont_send_proposal": END})
graph_builder.add_conditional_edges("send_proposal_node", tools_condition, {"tools": "proposal_tool_node", END: END})
graph_builder.add_edge("proposal_tool_node", "send_proposal_node")

graph = graph_builder.compile()

In [ ]:
# Display the graph

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Invoke the graph with website and company name

state = {"website": "https://www.askhumans.com", "company": "AskHumans"}
final_state = await graph.ainvoke(state)